Need to build the Transversional CNOT and CNOT gate

This let's use create logical CNOT gate 

Implement a logical CNOT as applying a CNOT on each corresponding physical qubit pair after encoding. 

Encode block A (control) and Encode block B (target)

CNOTc→t​=(I⊗H)CZ(I⊗H). H gate on target before, CZ control to target, H gate after on target



Process flow : 

- Encoding circuit (logical qubit) 
    - 7 qubit color/Steane state injection [[7,1,3]] 

- QEC (Steane) to create memory 

Encode Circuit (logical qubit)

![Encoding Circuit](Encode_circ_(logical_qubit).png)


Distillation circuit 

![Distillation Circuit](distillation_d3.png)

Used later (if at all)

In [ ]:
### Distillation Code ###

#call Logical qubit (encoded / error corrected) 

#squin.cz(q[1], q[2])
        # squin.sqrt_x(q[0])
        # squin.sqrt_x(q[1])
        # squin.sqrt_x(q[4])
        # squin.cz(q[0],q[1])
        # squin.cz(q[2],q[3])
        # squin.sqrt_y(q[0])
        # squin.sqrt_y(q[3])
        # squin.cz(q[0],q[2])
        # squin.cz(q[3],q[4])
        # squin.sqrt_x_adj(q[0]) 
        # squin.cz(q[1],q[3])
        # squin.cz(q[0],q[4])
        # for i in range(5):
        #    squin.sqrt_x_adj(q[i])

In [ ]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [ ]:
def MSD_encoding(theta, phi,basis="z"):

    @squin.kernel
    def parameterized_MSD_encoding():
        q = squin.qalloc(7)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
    
    return parameterized_MSD_encoding

In [ ]:
import math

def H(q): # H gate up to global phase
    squin.u3(math.pi/2, 0.0, math.pi, q)

def logical_cnot_blockwise(qc, qt):   # qc, qt are length-7 lists of qubits (encoded blocks)
    for i in range(7):
        H(qt[i])
        squin.cz(qc[i], qt[i])
        H(qt[i])

In [ ]:
@squin.kernel
def two_block_logical_cnot(theta_c, phi_c, theta_t, phi_t):
    qc = squin.qalloc(7)
    qt = squin.qalloc(7)

    MSD_encoding(theta_c, phi_c)()(qc)  
    MSD_encoding(theta_t, phi_t)()(qt)

    logical_cnot_blockwise(qc, qt)